# Scipy

In [ ]:
from IPython.display import Image, HTML
import numpy as np
from numpy import ma
import matplotlib.pyplot as plt
%matplotlib inline

**Scipy** can is a library (but can be thought more as a scientific python distribution) that is built on top of Numpy and
and provides a large set of standard scientific computing algorithms, in particular: 

* Special functions ([scipy.special](http://docs.scipy.org/doc/scipy/reference/special.html))
* Integration ([scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html))
* Optimization ([scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html))
* Interpolation ([scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html))
* Fourier Transforms ([scipy.fftpack](http://docs.scipy.org/doc/scipy/reference/fftpack.html))
* Signal Processing ([scipy.signal](http://docs.scipy.org/doc/scipy/reference/signal.html))
* Linear Algebra ([scipy.linalg](http://docs.scipy.org/doc/scipy/reference/linalg.html))
* Sparse Eigenvalue Problems ([scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html))
* Statistics ([scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html))
* Multi-dimensional image processing ([scipy.ndimage](http://docs.scipy.org/doc/scipy/reference/ndimage.html))
* File IO ([scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html))

We'll only have a look at some of these sub-packages, more directly relevant to **analyzing** data
   
   
+ **scipy.stats**: standard continuous and discrete probability distributions (density functions, samplers, ...), various statistical tests, and more descriptive statistics. 
+ **scipy.interpolate**: 1D and 2D interpolation


In [ ]:
HTML("<iframe src='http://scipy.org/' width=1000 height=500></iframe>")

In [ ]:
import scipy

## Statistics: Scipy.stats

In [ ]:
from scipy import stats

In [ ]:
dir(stats)

### Distributions and fitting distributions with Scipy stats

#### distributions in scipy 

see some references:     
+ [distributions in scipy by John D. Cook](http://www.johndcook.com/distributions_scipy.html)
+ [Probability distribution parameterizations in SciPy](http://www.johndcook.com/blog/2010/02/03/statistical-distributions-in-scipy/)

In [ ]:
stats.distributions.

In [ ]:
# create a (continous) random variable with normal distribution
Y = stats.distributions.norm()

In [ ]:
Y

In [ ]:
x = np.linspace(-5,5,100)

fig, axes = plt.subplots(3,1, sharex=True, figsize=(10,10))

# plot the probability distribution function (PDF)
axes[0].plot(x, Y.pdf(x))
axes[0].set_title('PDF')

# plot the commulative distributin function (CDF)
axes[1].plot(x, Y.cdf(x));
axes[1].set_title('CDF')

# plot histogram of 1000 random realizations of the variable Y ~ N(0,1)
axes[2].hist(Y.rvs(size=1000), bins=20);
axes[2].set_title('Random Variable');

#### Fitting data to a particular distribution

An recurring statistical problem is finding estimates of the relevant parameters that correspond to the distribution that best represents our data. In **parametric** inference, we specify *a priori* a suitable distribution, then choose the parameters that best fit the data.

We're going to see how to do that in Python using two methods:

* The **Method of moments** chooses the parameters so that the sample moments (typically the sample mean and variance) match the theoretical moments of our chosen distribution.
* The **Maximum likelihood** chooses the parameters to maximize the likelihood, which measures how likely it is to observe our given sample given the parameters. This is effectively done by **optimization**

A real life example: Monthly rainfall amounts at Auckland Airport station

In [ ]:
import pandas as pd; print(pd.__version__)

In [ ]:
data = pd.read_excel('./data/AKL_aero_rain_monthly.xlsx', sheetname='AKL',index_col=1)

In [ ]:
data.head()

let's look at the empirical distribution (thanks to Pandas)

In [ ]:
data['rain'].hist(normed=True, bins=20)

There are a few possible choices, but one suitable alternative is the [**gamma distribution**](http://en.wikipedia.org/wiki/Gamma_distribution):

There are three different parametrizations in common use, we're gonna use the the one below, with shape parameter $\alpha$ and an inverse scale parameter $\beta$ called a *rate parameter*.

<div style="font-size: 120%;">  
$$x \sim \text{Gamma}(\alpha, \beta) = \frac{\beta^{\alpha}x^{\alpha-1}e^{-\beta x}}{\Gamma(\alpha)}$$
</div>

##### Method of moments

The ***method of moments*** simply assigns the empirical mean and variance to their theoretical counterparts, so that we can solve for the parameters.

So, for the gamma distribution, it turns out (see the relevant section in [the wikipedia article](http://en.wikipedia.org/wiki/Gamma_distribution)) that the mean and variance are:

<div style="font-size: 120%;">  
$$ \hat{\mu} = \alpha \beta $$
$$ \hat{\sigma}^2 = \alpha \beta^2 $$
</div>

So, if we solve for $\alpha$ and $\beta$, using the **sample** mean ($\bar{X}$) and variance ($S^2$), we can use a gamma distribution to describe our data:

<div style="font-size: 120%;">  
$$ \alpha = \frac{\bar{X}^2}{S^2}, \, \beta = \frac{S^2}{\bar{X}} $$
</div>

first step: we calculate the **sample mean and variance**, using pandas convenience methods 

In [ ]:
precip_mean = data['rain'].mean() # sample mean 
precip_var = data['rain'].var() # sample variance

In [ ]:
print("mean: {:4.2f}\nvariance: {:4.2f}".format(precip_mean, precip_var))

second step: we calculate the parameters $\alpha$ and $\beta$ using the relations above

In [ ]:
alpha_mom = precip_mean ** 2 / precip_var
beta_mom = precip_var / precip_mean

We import the ```gamma``` function from the scipy.stats.distributions sub-package  
The implementation of the Gamma function in scipy requires a *location* parameter, left to zero

In [ ]:
from scipy.stats.distributions import gamma 

In [ ]:
gmom = gamma(alpha_mom, 0, beta_mom)

#or gmom = gamma(alpha_mom, scale=beta_mom)

In [ ]:
plt.hist(data['rain'], normed=True, bins=20)
plt.plot(np.linspace(0, 300, 100), gmom.pdf(np.linspace(0, 300, 100)), lw=3, c='r')

##### Maximum Likelihood Estimates using the .fit() method

In [ ]:
shape,loc,scale = gamma.fit(data['rain'])

In [ ]:
shape, loc, scale

In [ ]:
alpha_mom, beta_mom

In [ ]:
gmle = gamma(shape,loc,scale)

In [ ]:
f, (ax0, ax1) = plt.subplots(1,2,figsize=(13,5))

rmax = 300; N=100

ax0.hist(data['rain'], normed=True, bins=20, color='0.6')
ax0.plot(np.linspace(0, rmax, N), gmle.pdf(np.linspace(0, rmax, N)), 'b-', lw=2, label='MLE')
ax0.plot(np.linspace(0, rmax, N), gmom.pdf(np.linspace(0, rmax, N)), 'r-', lw=2, label='Moments')
ax0.legend()
ax0.set_title('Histogram and fitted PDFs')
ax1.plot(np.linspace(0, rmax, N), gmle.cdf(np.linspace(0, rmax, N)), 'b-', lw=2, label='MLE')
ax1.plot(np.linspace(0, rmax, N), gmom.cdf(np.linspace(0, rmax, N)), 'r-', lw=2, label='Moments')
ax1.legend()
ax1.set_title('CDFs');

**Goodness of fit** tests are available in scipy.stats, through e.g. 

+ the Kolmogorov-Smirnoff test (`scipy.stats.kstest`)
+ the Anderson-Darling test (`scipy.stats.anderson`)

In [ ]:
from scipy.stats import kstest

In [ ]:
kstest(data['rain'], 'gamma', (shape, loc, scale))

In [ ]:
kstest(data['rain'], 'gamma', (alpha_mom, 0, beta_mom))

### Non Parametric Density estimation: Kernel Density Estimation

In some instances, we may not be interested in the parameters of a particular distribution of data, but just a smoothed representation of the data. In this case, we can estimate the distribution *non-parametrically* (i.e. making no assumptions about the form of the underlying distribution) using [Kernel Density Estimation](http://en.wikipedia.org/wiki/Kernel_density_estimation).

If you are interested into an excellent discussion of the various options in Python to perform Kernel Density Estimation, have a look at this [post](http://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/) on Jake VanderPlas blog [Pythonic Perambulations](http://jakevdp.github.io/). Some more stuff on KDE is available [here](http://www.mglerner.com/blog/?p=28) from Michael Lerner.  

In a nutschell, (Gaussian) Kernel density estimation works 'simply' by summing up Gaussian functions (PDFs) centered on each data point. Each Gaussian function is characterized by a location parameter (the value of the data point), and a scale parameter ($\sigma$) which is related to the 'bandwith' of your (Gaussian) kernel.

In [ ]:
# Some random data
y = np.random.random(15) * 10
y

In [ ]:
x = np.linspace(0, 10, 100)
# Smoothing parameter
s = 0.4
# Calculate the kernels
kernels = np.transpose([stats.distributions.norm.pdf(x, yi, s) for yi in y])
plt.plot(x, kernels, 'k:', lw=1)
plt.plot(x, kernels.sum(1), lw=1.5)
plt.plot(y, np.zeros(len(y)), 'ro', ms=10)

#### Univariate KDE

Kernel Density Estimation is implemented in scipy.stats.kde by [scipy.stats.kde.gaussian_kde](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html)

In [ ]:
from scipy.stats.kde import gaussian_kde

In [ ]:
f, ax = plt.subplots(figsize=(7,7))

xgrid = np.linspace(data['rain'].min(), data['rain'].max(), 100)
density = gaussian_kde(data['rain']).evaluate(xgrid)

ax.hist(data['rain'], bins=20, normed=True, label='data', alpha=.6)
ax.plot(xgrid, density, 'r-', lw=2, label='KDE')
ax.legend()

#### Multivariate Kernel Density Estimation

An example in 2 dimensions (from [https://gist.github.com/endolith/1035069](https://gist.github.com/endolith/1035069))

In [ ]:
# Create some dummy data
rvs = np.append(stats.norm.rvs(loc=2,scale=1,size=(200,1)),
                stats.norm.rvs(loc=1,scale=3,size=(200,1)),
                axis=1)
 
kde = stats.kde.gaussian_kde(rvs.T)
 
# Regular grid to evaluate kde upon
x_flat = np.r_[rvs[:,0].min():rvs[:,0].max():128j]
y_flat = np.r_[rvs[:,1].min():rvs[:,1].max():128j]

x,y = np.meshgrid(x_flat,y_flat)

grid_coords = np.append(x.reshape(-1,1),y.reshape(-1,1),axis=1)
 
z = kde(grid_coords.T)
z = z.reshape(128,128)
 
# Plot
f, ax = plt.subplots(figsize=(8,8))
ax.scatter(rvs[:,0],rvs[:,1],alpha=0.8,color='white')
im = ax.imshow(z,aspect=x_flat.ptp()/y_flat.ptp(),origin='lower',\
          extent=(rvs[:,0].min(),rvs[:,0].max(),rvs[:,1].min(),rvs[:,1].max()),\
          cmap=plt.get_cmap('spectral'))
plt.colorbar(im, shrink=0.8);

### Statistical tests 

One commonly used parametric test is the *Student t-test*, which determines whether two independant samples have - statistically - different means, under the assumption that the two samples are normally distributed. The Null hypothesis is that the two samples have **identical** means.

In [ ]:
stats.ttest_ind?

In [ ]:
X = stats.distributions.norm(loc=10, scale=10)
Y = stats.distributions.norm(loc=1, scale=10)

Xrvs = X.rvs(size=(1000))
Yrvs = Y.rvs(size=(1000))

In [ ]:
f, ax = plt.subplots(figsize=(6,5))
ax.hist(Xrvs,histtype='stepfilled', bins=20, color='coral', alpha=.6, label=r'$\mu = 5$')
ax.hist(Yrvs,histtype='stepfilled', bins=20, color='steelblue', alpha=.6, label=r'$\mu = 3$')
ax.legend();

In [ ]:
t, p = stats.ttest_ind(Xrvs, Yrvs)
print("""\n
T statistics = {0} ~= {0:4.2f}
P-value = {1} ~= {1:4.2f}
""".format(t, p))

### Correlation (Pearson's R)

In [ ]:
from scipy.stats import pearsonr

We're gonna apply that to Time-Series of NINO3.4 and the SOI 

The NINO indices come from [http://www.cpc.ncep.noaa.gov/data/indices/ersst3b.nino.mth.81-10.ascii](http://www.cpc.ncep.noaa.gov/data/indices/ersst3b.nino.mth.81-10.ascii)

The SOI is in the `data` directory

In [ ]:
import os
from datetime import datetime
from dateutil import parser
import pandas as pd

In [ ]:
nino_url = "http://www.cpc.ncep.noaa.gov/data/indices/ersst3b.nino.mth.81-10.ascii"

In [ ]:
nino = pd.read_table(nino_url, sep='\s+', engine='python')
# if not working try: 
# nino = pd.read_table('../data/ersst3b.nino.mth.81-10.ascii', sep='\s+', engine='python')
dates = [datetime(x[0], x[1], 1) for x in zip(nino['YR'].values, nino['MON'].values)]
nino.index = dates

In [ ]:
nino.tail()

In [ ]:
soipath = './data'
def get_SOI(soipath, start_date='1950-01-01'):
    soi = pd.read_csv(os.path.join(soipath,'NICO_NIWA_SOI_1941_2010.csv'), index_col=0)
    soi = soi.stack()
    soi = soi.dropna()
    dates = [parser.parse("%s-%s-%s" % (str(int(x[0])), x[1], "1")) for x in soi.index]
    soidf = pd.DataFrame(soi.values, index=dates, columns=['SOI'])
    soidf = soidf.truncate(before=start_date)
    return soidf

creates another DataFrame containing the SOI

In [ ]:
df = get_SOI(soipath)

In [ ]:
df.tail()

Will automaticall **ALIGN** the DataFrames, see [Pandas.ipynb](./Pandas.ipynb)

In [ ]:
df['nino'] = nino['ANOM.3']

In [ ]:
df.dropna(inplace=True)

In [ ]:
r, p = pearsonr(df['SOI'], df['nino'])

print("R = {0:<4.2f}, p-value = {1:<4.2f}".format(r,p))

In [ ]:
df = pd.DataFrame({'SOI':soi['SOI'],'nino':nino['ANOM.3']}, index=nino.index)

In [ ]:
df.head()

In [ ]:
df.to_csv('./data/soi_nino.csv')

## Interpolation: Scipy.interpolate

There are several interfaces for performing 1D, 2D or N Dimensional interpolation using scipy.

For more on that I refer you to 

+ [http://docs.scipy.org/doc/scipy/reference/interpolate.html](http://docs.scipy.org/doc/scipy/reference/interpolate.html)
+ [http://wiki.scipy.org/Cookbook/Interpolation](http://wiki.scipy.org/Cookbook/Interpolation)

Here we're going to see one simple example in 1D and 2D using [Radial Basis Functions](http://en.wikipedia.org/wiki/Radial_basis_function) using the default (Multiquadric)

### 1D interpolation

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
x = np.linspace(0, 10, 10)
y = np.sin(x)

In [ ]:
x

In [ ]:
plt.plot(x,y,'ro')

In [ ]:
f1 = interp1d(x, y) # default is linear

In [ ]:
f2 = interp1d(x, y, kind='cubic') # and we're gonna try cubic interpolation for comparison

In [ ]:
xnew = np.linspace(0, 10, 50)

In [ ]:
ylin = f1(xnew)
ycub = f2(xnew)

In [ ]:
f, ax = plt.subplots(figsize=(10,6))
ax.plot(xnew, ylin, 'b.', ms=8, label='linear')
ax.plot(xnew, ycub, 'gx-', ms=10, label='cubic')
ax.plot(x,y,'ro-', label='data')
ax.legend(loc='lower left')

### 2D interpolation using Radial Basis Functions

In [ ]:
from scipy.interpolate import Rbf

In [ ]:
# defines a function of X and Y
def func(x,y):
    return x*np.exp(-x**2-y**2)

In [ ]:
np.random.seed(1) # we 'fix' the generation of random numbers so that we've got consistent results

x = np.random.uniform(-2., 2., 100)
y = np.random.uniform(-2., 2., 100)

z = func(x,y)

ti = np.linspace(-2.0, 2.0, 100)

XI, YI = np.meshgrid(ti, ti) # meshgrid creates uniform 2D grids from 1D vectors

# use RBF
rbf = Rbf(x, y, z, epsilon=2) # instantiates the interpolator
# you might want to play with the epsilon optional parameter 

ZI = rbf(XI, YI) # interpolate on grid

In [ ]:
# this is the 'True' field, the result of the function evaluated on a regular grid
true_Z = func(XI, YI)

In [ ]:
# plot the result
f, ax = plt.subplots(figsize=(8,6))
im = ax.pcolor(XI, YI, ZI, cmap=plt.get_cmap('RdBu_r'))
ax.scatter(x, y, 50, z, cmap=plt.get_cmap('RdBu_r'), edgecolor='.5')
ax.set_title('RBF interpolation - multiquadrics')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
plt.colorbar(im, orientation='vertical', pad=0.06); 

In [ ]:
# plot the difference between the 'true' field and the interpolated 
f, ax = plt.subplots(figsize=(8,6))
im = ax.pcolor(XI, YI, ZI - true_Z, cmap=plt.get_cmap('RdBu_r'), vmin=-1e-3, vmax=1e-3)
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
plt.colorbar(im, orientation='vertical', pad=0.06); 

## Curve fitting with scipy.optimize

In this case you want to fit noisy, e.g. experimental, data to **a specific function**, with *unknown parameters*

In [ ]:
from scipy.optimize import curve_fit

The algorithm uses the [Levenberg-Marquardt](http://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm) algorithm to perform non-linear least-square optimization

In [ ]:
def fitFunc(t, a, b, c):
    """ 
    defines the function
    takes 3 parameters: a, b and c
    """
    return a*np.exp(-b*t) + c

In [ ]:
### defines the evaluation domain
t = np.linspace(0,4,50)

### defines the paramaters 
a = 5.0
b = 1.5
c = 0.5

### create the response
temp = fitFunc(t, a, b, c)

### add some noise to simulate "real world observations" such as experimental data
noisy = temp + 0.4 * np.random.normal(size=len(temp)) 

### use curve_fit to estimate the parameters and the covariance matrices
fitParams, fitCovariances = curve_fit(fitFunc, t, noisy)

afit, bfit, cfit = tuple(fitParams)

print("\nEstimated parameters\na: {0:<4.2f}, b: {1:<4.2f}, c: {2:<4.2f}\n\n".format(afit, bfit, cfit))

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
ax.set_ylabel(u'Temperature (\xb0C)', fontsize = 16)
ax.set_xlabel('time (s)', fontsize = 16)
ax.set_xlim(0,4.1)
# plot the data as red circles with vertical errorbars
ax.errorbar(t, noisy, fmt = 'ro', yerr = 0.2)
# now plot the best fit curve 

ax.plot(t, fitFunc(t, afit, bfit, cfit),'k-', lw=2)

# and plot the +- 1 sigma curves
# (the square root of the diagonal covariance matrix  
# element is the uncertainty on the fit parameter.)

sigma_a, sigma_b, sigma_c = np.sqrt(fitCovariances[0,0]), \
np.sqrt(fitCovariances[1,1]), \
np.sqrt(fitCovariances[2,2])

ax.plot(t, fitFunc(t, afit + sigma_a, bfit - sigma_b, cfit + sigma_c), 'b-')
ax.plot(t, fitFunc(t, afit - sigma_a, bfit + sigma_b, cfit - sigma_c), 'b-');

### A little exercise of curve fitting 

solution available at [http://nbviewer.ipython.org/gist/nicolasfauchereau/79131703a0340e1ed82a](http://nbviewer.ipython.org/gist/nicolasfauchereau/79131703a0340e1ed82a)

In [ ]:
import pandas as pd

In [ ]:
data= pd.read_excel('./data/rainfall_calibration.xlsx', sheetname='Sheet1')

In [ ]:
data.head()

In [ ]:
f, ax = plt.subplots()
ax.plot(data['Rainfall'], data['Level'], 'b', lw=1.5)
ax.plot(data['Rainfall'], data['Level'], 'ro')
ax.set_xlabel('Rainfall')
ax.set_ylabel('Level')

...